In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Connect to workspace
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="2d3821fb-1fc0-4f0e-b514-0bdca749349b",
    resource_group_name="ml-rg-demo",
    workspace_name="ml-workspace-demo"
)

print("✅ Connected to workspace:", ml_client.workspace_name)


In [ ]:
from azure.ai.ml.entities import Data

my_data = Data(
    name="walmart-data",
    description="Demo dataset for walmart prediction",
    path="Walmart.csv",  # relative to notebook folder
    type="uri_file"
)

ml_client.data.create_or_update(my_data)
print("✅ Dataset registered")

In [27]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="walmart-env",
    version="2",   # must include version
    conda_file="conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04"
)
ml_client.environments.create_or_update(env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'walmart-env', 'description': None, 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/2d3821fb-1fc0-4f0e-b514-0bdca749349b/resourceGroups/ml-rg-demo/providers/Microsoft.MachineLearningServices/workspaces/ml-workspace-demo/environments/walmart-env/versions/2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ml-220161-compute/code/Users/upg1756791740_1756791745087', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7ebb0ef3eaa0>, 'serialize': <msrest.serialization.Serializer object at 0x7ebb0ed5bfd0>, 'version': '2', 'conda_file': {'dependencies': ['python=3.10', 'scikit-learn', 'pandas', {'pip': ['joblib', 'mlfl

In [ ]:
from azure.ai.ml import command, Input

job = command(
    code=".",  # folder containing train.py
    command="python train.py --data ${{inputs.data}} --n_estimators 200",
    inputs={"data": Input(type="uri_file", path="walmart-data:1")},
    environment="walmart-env:2",
    compute="ml-220161-compute",
    experiment_name="walmart-mlflow-training"
)

returned_job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(returned_job.name)


In [ ]:
from azure.ai.ml.entities import Model

model = Model(
    path=f"azureml://jobs/{returned_job.name}/outputs/artifacts/model.pkl",
    name="walmart-rf-model",
    description="Random Forest model trained on Walmart dataset"
)

ml_client.models.create_or_update(model)
print("✅ Model registered")
